### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import bamboolib as bam

### Load the Loan Data

In [2]:
loan_data = pd.read_csv(r'C:\Users\jjsun\OneDrive\Desktop\pythonprojects\LendingClub-Loan_Default-Prediction\data\raw\Loan_status_2007-2020Q3.gzip')

In [3]:
# print([column for column in df_scan.columns])
# features = [feature for feature in df_scan.columns if feature not in ['Unnamed: 0', 'id']]
loan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925493 entries, 0 to 2925492
Columns: 142 entries, Unnamed: 0 to debt_settlement_flag
dtypes: float64(106), int64(1), object(35)
memory usage: 3.1+ GB


Loading in the loan data shows that we have almost three million observations and 142 features. Thirty-five of the columns are categorical (object type) and 107 columns are numerical (float64 and int 64).

In [4]:
print(loan_data.head(5))

   Unnamed: 0       id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0           0  1077501     5000.0       5000.0           4975.0   36 months   
1           1  1077430     2500.0       2500.0           2500.0   60 months   
2           2  1077175     2400.0       2400.0           2400.0   36 months   
3           3  1076863    10000.0      10000.0          10000.0   36 months   
4           4  1075358     3000.0       3000.0           3000.0   60 months   

  int_rate  installment grade sub_grade  ... hardship_start_date  \
0   10.65%       162.87     B        B2  ...                 NaN   
1   15.27%        59.83     C        C4  ...                 NaN   
2   15.96%        84.33     C        C5  ...                 NaN   
3   13.49%       339.31     C        C1  ...                 NaN   
4   12.69%        67.79     B        B5  ...                 NaN   

  hardship_end_date payment_plan_start_date  hardship_length hardship_dpd  \
0               NaN                    

In [5]:
print(loan_data.describe())

         Unnamed: 0     loan_amnt   funded_amnt  funded_amnt_inv  \
count  2.925493e+06  2.925492e+06  2.925492e+06     2.925492e+06   
mean   8.697419e+04  1.535878e+04  1.535470e+04     1.534005e+04   
std    8.048842e+04  9.478383e+03  9.477145e+03     9.480634e+03   
min    0.000000e+00  5.000000e+02  5.000000e+02     0.000000e+00   
25%    3.284100e+04  8.000000e+03  8.000000e+03     8.000000e+03   
50%    6.892500e+04  1.300000e+04  1.300000e+04     1.300000e+04   
75%    1.072940e+05  2.000000e+04  2.000000e+04     2.000000e+04   
max    4.210940e+05  4.000000e+04  4.000000e+04     4.000000e+04   

        installment    annual_inc           dti   delinq_2yrs  fico_range_low  \
count  2.925492e+06  2.925488e+06  2.922384e+06  2.925463e+06    2.925492e+06   
mean   4.523914e+02  7.993727e+04  1.929648e+01  2.897012e-01    7.005110e+02   
std    2.729478e+02  1.117476e+05  1.571977e+01  8.436819e-01    3.376565e+01   
min    4.930000e+00  0.000000e+00 -1.000000e+00  0.000000e+00  

### Missing Values

In [8]:
missing = pd.concat([loan_data.isnull().sum().sort_values(ascending = False), 100 * loan_data.isnull().mean().sort_values(ascending = False)], axis=1)
missing.columns=['count', '%']
print(missing)

                        count          %
hardship_loan_status  2782082  95.097886
hardship_reason       2781861  95.090332
hardship_status       2781858  95.090229
hardship_dpd          2781856  95.090161
hardship_type         2781855  95.090127
...                       ...        ...
last_fico_range_low         1   0.000034
policy_code                 1   0.000034
funded_amnt                 1   0.000034
id                          0   0.000000
Unnamed: 0                  0   0.000000

[142 rows x 2 columns]


### Categorical Features

In [9]:
print(loan_data.select_dtypes('object').head())

        id        term int_rate grade sub_grade                 emp_title  \
0  1077501   36 months   10.65%     B        B2                       NaN   
1  1077430   60 months   15.27%     C        C4                     Ryder   
2  1077175   36 months   15.96%     C        C5                       NaN   
3  1076863   36 months   13.49%     C        C1       AIR RESOURCES BOARD   
4  1075358   60 months   12.69%     B        B5  University Medical Group   

  emp_length home_ownership verification_status   issue_d  ...  \
0  10+ years           RENT            Verified  Dec-2011  ...   
1   < 1 year           RENT     Source Verified  Dec-2011  ...   
2  10+ years           RENT        Not Verified  Dec-2011  ...   
3  10+ years           RENT     Source Verified  Dec-2011  ...   
4     1 year           RENT     Source Verified  Dec-2011  ...   

  sec_app_earliest_cr_line hardship_flag hardship_type hardship_reason  \
0                      NaN             N           NaN            

In [10]:
loan_data.select_dtypes('object').columns

Index(['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'url', 'purpose', 'title', 'zip_code',
       'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status',
       'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d',
       'application_type', 'verification_status_joint',
       'sec_app_earliest_cr_line', 'hardship_flag', 'hardship_type',
       'hardship_reason', 'hardship_status', 'hardship_start_date',
       'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status',
       'debt_settlement_flag'],
      dtype='object')

### Loan Terms
The loan duration ('term' feature) is a str object. It would be better to have this as a numeric feature, so let's change that.

In [11]:
# Find null entries for loan term
print(loan_data[loan_data.term.isnull()])
# Drop useless row for loans that do not meet the credit policy
loan_data.drop(loan_data[loan_data.term.isnull()].index, inplace = True)
# Extract the loan term length and convert to int64
loan_data['term'] = loan_data['term'].str[1:3].astype('int64')

       Unnamed: 0                                        id  loan_amnt  \
39786       39786  Loans that do not meet the credit policy        NaN   

       funded_amnt  funded_amnt_inv term int_rate  installment grade  \
39786          NaN              NaN  NaN      NaN          NaN   NaN   

      sub_grade  ... hardship_start_date hardship_end_date  \
39786       NaN  ...                 NaN               NaN   

      payment_plan_start_date  hardship_length hardship_dpd  \
39786                     NaN              NaN          NaN   

      hardship_loan_status orig_projected_additional_accrued_interest  \
39786                  NaN                                        NaN   

      hardship_payoff_balance_amount hardship_last_payment_amount  \
39786                            NaN                          NaN   

      debt_settlement_flag  
39786                  NaN  

[1 rows x 142 columns]


In [12]:
# Confirm that there are no more null loan term observations
print(loan_data[loan_data.term.isnull()].index)
# Rename the column to specify the units
loan_data.rename(columns={'term':'term (months)'})

Int64Index([], dtype='int64')


,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term (months),int_rate,installment,grade,sub_grade,...,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,1077501,5000.0,5000.0,4975.0,36,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,1077430,2500.0,2500.0,2500.0,60,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,2,1077175,2400.0,2400.0,2400.0,36,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,3,1076863,10000.0,10000.0,10000.0,36,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,4,1075358,3000.0,3000.0,3000.0,60,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925488,105446,102556443,24000.0,24000.0,24000.0,60,23.99%,690.30,E,E2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2925489,105447,102653304,10000.0,10000.0,10000.0,36,7.99%,313.32,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2925490,105448,102628603,10050.0,10050.0,10050.0,36,16.99%,358.26,D,D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2925491,105449,102196576,6000.0,6000.0,6000.0,36,11.44%,197.69,B,B4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


### Interest Rate
The interest rate is also string data that should be converted into numeric float data.

In [13]:
# Find null entries for loan term
print(loan_data[loan_data.int_rate.isnull()])
# There are no null observations for interest rate
# Extract the loan term length and convert to int64
loan_data['int_rate'] = loan_data['int_rate'].str.strip('%').astype('float')/100

Empty DataFrame
Columns: [Unnamed: 0, id, loan_amnt, funded_amnt, funded_amnt_inv, term, int_rate, installment, grade, sub_grade, emp_title, emp_length, home_ownership, annual_inc, verification_status, issue_d, loan_status, pymnt_plan, url, purpose, title, zip_code, addr_state, dti, delinq_2yrs, earliest_cr_line, fico_range_low, fico_range_high, inq_last_6mths, mths_since_last_delinq, mths_since_last_record, open_acc, pub_rec, revol_bal, revol_util, total_acc, initial_list_status, out_prncp, out_prncp_inv, total_pymnt, total_pymnt_inv, total_rec_prncp, total_rec_int, total_rec_late_fee, recoveries, collection_recovery_fee, last_pymnt_d, last_pymnt_amnt, next_pymnt_d, last_credit_pull_d, last_fico_range_high, last_fico_range_low, collections_12_mths_ex_med, mths_since_last_major_derog, policy_code, application_type, annual_inc_joint, dti_joint, verification_status_joint, acc_now_delinq, tot_coll_amt, tot_cur_bal, open_acc_6m, open_act_il, open_il_12m, open_il_24m, mths_since_rcnt_il, to

In [ ]:
# id feature changed to integer
# loan amount changed to integer
# funded amount changed to integer
# funded amount inv kept as float
# grade changed to category
# subgrade changed to category
# employment title missing values filled with No Employment, type changed to string
# employment length missing values filled with No Employment, type changed to category
# home ownership type changed to category
# annual income missing values filled with zero
# verification status type changed to category
# issue date changed to datetime
# loan status changed to category
# payment plan changed to category (drop this), renamed and old column dropped
# url changed to string
# purpose changed to category
# title missing values filled with No Title
# title changed to string
# zip code missing values filled with mode, changed to category
# address state changed to category
# DTI missing values filled with mode
# Deliquency 2 years missing values REWORK
# Earliest credit line missing values REWORK
# FICO range low and high changed to integer
# Inquiry Last 6 Months missing values REWORK 

In [14]:
# df_scan.fico_range_high.hist()
# df_scan.purpose.value_counts()

loan_data['id'] = pd.to_numeric(loan_data['id'], downcast='integer', errors='coerce')
loan_data['loan_amnt'] = pd.to_numeric(loan_data['loan_amnt'], downcast='integer', errors='coerce')
loan_data['funded_amnt'] = pd.to_numeric(loan_data['funded_amnt'], downcast='integer', errors='coerce')
# fix funded_amnt_inv
loan_data['grade'] = loan_data['grade'].astype('category')
loan_data['sub_grade'] = loan_data['sub_grade'].astype('category')

loan_data[['emp_title']] = loan_data[['emp_title']].fillna('No Employment')
loan_data['emp_title'] = loan_data['emp_title'].astype('string')
loan_data[['emp_length']] = loan_data[['emp_length']].fillna('No Employment')
loan_data['emp_length'] = loan_data['emp_length'].astype('category')
loan_data['home_ownership'] = loan_data['home_ownership'].astype('category')
loan_data[['annual_inc']] = loan_data[['annual_inc']].fillna(0)
loan_data['verification_status'] = loan_data['verification_status'].astype('category')
loan_data['issue_d'] = pd.to_datetime(loan_data['issue_d'], infer_datetime_format=True)
loan_data['loan_status'] = loan_data['loan_status'].astype('category')
loan_data['payment_plan'] = loan_data['pymnt_plan'].astype('category')
loan_data = loan_data.drop(columns=['pymnt_plan'])
loan_data['url'] = loan_data['url'].astype('string')
loan_data['purpose'] = loan_data['purpose'].astype('category')
loan_data[['title']] = loan_data[['title']].fillna('No Title')
loan_data['title'] = loan_data['title'].astype('string')
loan_data[['zip_code']] = loan_data[['zip_code']].fillna(loan_data[['zip_code']].mode().iloc[0])
loan_data['zip_code'] = loan_data['zip_code'].astype('category')
loan_data['addr_state'] = loan_data['addr_state'].astype('category')
loan_data[['dti']] = loan_data[['dti']].fillna(loan_data[['dti']].mode().iloc[0])
loan_data[['delinq_2yrs']] = loan_data[['delinq_2yrs']].fillna(loan_data[['delinq_2yrs']].mode().iloc[0])
loan_data['delinq_2yrs'] = pd.to_numeric(loan_data['delinq_2yrs'], downcast='integer', errors='coerce')
loan_data[['earliest_cr_line']] = loan_data[['earliest_cr_line']].fillna(loan_data[['earliest_cr_line']].mode().iloc[0])
loan_data['earliest_cr_line'] = pd.to_datetime(loan_data['earliest_cr_line'], infer_datetime_format=True)
for column_name in ['fico_range_low', 'fico_range_high']:
    loan_data[column_name] = loan_data[column_name].astype('int')
loan_data[['inq_last_6mths']] = loan_data[['inq_last_6mths']].fillna(loan_data[['inq_last_6mths']].mode().iloc[0])
loan_data['inq_last_6months'] = pd.to_numeric(loan_data['inq_last_6mths'], downcast='integer', errors='coerce')
loan_data = loan_data.drop(columns=['inq_last_6mths'])

loan_data

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,payment_plan,inq_last_6months
0,0,1077501,5000,5000,4975.0,36,0.1065,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,1
1,1,1077430,2500,2500,2500.0,60,0.1527,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,5
2,2,1077175,2400,2400,2400.0,36,0.1596,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,2
3,3,1076863,10000,10000,10000.0,36,0.1349,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,1
4,4,1075358,3000,3000,3000.0,60,0.1269,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925488,105446,102556443,24000,24000,24000.0,60,0.2399,690.30,E,E2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,1
2925489,105447,102653304,10000,10000,10000.0,36,0.0799,313.32,A,A5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,0
2925490,105448,102628603,10050,10050,10050.0,36,0.1699,358.26,D,D1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,1
2925491,105449,102196576,6000,6000,6000.0,36,0.1144,197.69,B,B4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,n,0
